In [ ]:
import geemap
import ee
from Monarch import user_gee

Map = geemap.Map()
Map

In [ ]:
point = Map.draw_last_feature.geometry()

In [ ]:
image = images = ee.ImageCollection("COPERNICUS/S2").filterDate('2020-06-21', '2021-06-24').filterBounds(point)\
                    .sort("CLOUDY_PIXEL_PERCENTAGE", False).mosaic().clip(point)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 3000, 'min': 0}, "imags")

In [ ]:
image = images = ee.ImageCollection("COPERNICUS/S2").filterDate('2020-06-21', '2021-06-24').filterBounds(point)\
                    .sort("CLOUDY_PIXEL_PERCENTAGE", False).mosaic().clip(point)

In [ ]:
image.reproject(**{'crs': 'epsg:4326', 'scale': 10})

In [ ]:
NDVI = image.normalizedDifference(['B8', 'B4'])
Map.addLayer(NDVI, {}, 'ndvi')

In [ ]:
NDVI.shape

In [ ]:
geemap.download_ee_image()

In [ ]:
def clip_dow_merge(geo: ee.Geometry, image: ee.Image, outfile: str, scale: int,
                   crs='epsg:4326', sep=0.25):
    """

    Args:
        geo: ee.Geometry, 需要下载的区域矢量几何
        image: ee.Image, 需要下载的影像
        outfile: str, 输出文件路径和名称，不需要文件后缀，下载的影响默认后缀为tif
        scale: int, 下载时的像元大小
        crs: str, 下载影像的投影，默认为 'epsg:4326' wgs1984投影
        sep: float, 单波段10m分辨率像元的影像裁剪大小(单位：经纬度)，默认为0.25
    Returns: None

    """
    import os
    import numpy as np
    import rasterio
    from glob import glob
    from rasterio.merge import merge
    import shutil
    import geemap
    import math
    bounds = geo.bounds()
    bands = image.bandNames().size().getInfo()
    poy = np.array(bounds.coordinates().getInfo()[0])
    min_x = poy[:, 0].min()
    max_x = poy[:, 0].max()
    min_y = poy[:, 1].min()
    max_y = poy[:, 1].max()
    step = scale / 10 * sep / (int(math.sqrt(bands))+1)
    end_x = int((max_x - min_x) / step) + 1
    end_y = int((max_y - min_y) / step) + 1
    polys = []
    for i in range(end_y):
        y1 = min_y + step * i
        y2 = min_y + step * (i + 1)
        if y2 > max_y:
            y2 = max_y
        for j in range(end_x):
            x1 = min_x + step * j
            x2 = min_x + step * (j + 1)
            if x2 > max_x:
                x2 = max_x
            poly = ee.Geometry(ee.Geometry.Rectangle([float(x1), float(y1), float(x2), float(y2)]), None, False)
            polys.append(poly)
    if len(polys) > 1:
        print(f"分割成{len(polys)}份, 开始下载:")
        path = outfile+'_mk'
        t = 1
        if not os.path.exists(path):
            t = 0
            os.makedirs(path)
        for j, i in enumerate(polys):
            if not os.path.exists(path+f'/temp_{j}.tif'):
                if t:
                    clip_dow_merge(i, image, path + f'/temp_{j}', scale, sep=sep*0.5)
                else:
                    geemap.ee_export_image(image, path + f'/temp_{j}.tif', scale=scale, crs=crs, region=i)
            else:
                continue
        files = glob(path+"/*.tif")
        if len(files) == len(polys):
            src_files_to_mosaic = []
            for tif_f in files:
                src = rasterio.open(tif_f)
                src_files_to_mosaic.append(src)
            mosaic, out_trans = merge(src_files_to_mosaic)
            out_meta = src.meta.copy()
            out_meta.update({"driver": "GTiff",
                             "height": mosaic.shape[1],
                             "width": mosaic.shape[2],
                             "transform": out_trans,
                             })
            with rasterio.open(outfile+".tif", "w", **out_meta) as dest:
                dest.write(mosaic)
            for src in src_files_to_mosaic:
                src.close()
            shutil.rmtree(path)
        else:
            print('下载不完全！！')
    else:
        geemap.ee_export_image(image, outfile+'.tif', scale=scale, crs=crs, region=geo)
    print("download successful !!!")

In [ ]:
roi = Map.draw_last_feature.geometry()

In [ ]:
import os
from Monarch import clip_dow_merge
out_name = 'D:/Data/s2_img_ndvi'
while not os.path.exists(out_name+'.tif'):
    clip_dow_merge.clip_dow_merge(roi, NDVI, out_name, 10, 4)

In [ ]:
image = ee.ImageCollection("MODIS/006/MOD09GQ")\
            .filterDate("2015-01-01", "2015-01-02").select('sur_refl_b01')\
            .first()

In [ ]:
image.bandNames().size().getInfo()

In [ ]:
geemap.down